# Model Evaluation

Evaluate all wind speed prediction models using the evaluation framework.


In [7]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from eval_helpers import evaluate, plot_predictions, compare
from data.data_helpers import get_dataframe
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## Load Data


In [8]:
# Load data
df = get_dataframe()
df = df.dropna(subset=["target_next_hour"])

# Temporal split: 2018-2019 for training, 2020 for testing
train_df = df[df["datetime"] < "2020-01-01"].copy()
test_df = df[df["datetime"] >= "2020-01-01"].copy()

print(f"Training set: {len(train_df):,} samples")
print(f"Test set: {len(test_df):,} samples")

# Store datetime for plotting
test_datetime = test_df["datetime"].copy()


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Skipping existing file /Users/aboulmich/Projects/cis5200-project/data/era5_tehachapi_2018_H1.nc
Skipping existing file /Users/aboulmich/Projects/cis5200-project/data/era5_tehachapi_2018_H2.nc
Skipping existing file /Users/aboulmich/Projects/cis5200-project/data/era5_tehachapi_2019_H1.nc
Skipping existing file /Users/aboulmich/Projects/cis5200-project/data/era5_tehachapi_2019_H2.nc
Skipping existing file /Users/aboulmich/Projects/cis5200-project/data/era5_tehachapi_2020_H1.nc
Skipping existing file /Users/aboulmich/Projects/cis5200-project/data/era5_tehachapi_2020_H2.nc
Training set: 17,520 samples
Test set: 8,783 samples


## Evaluate Models

Evaluate each model and collect results.


In [ ]:
# Collect results from all models
results = []

# TODO: Add your models here
# Example:
# from models.model0 import PersistenceModel
# model0 = PersistenceModel()
# result0 = evaluate(model0, X_train_m0, y_train_m0, X_test_m0, y_test_m0, model_name="PersistenceBaseline")
# results.append(result0)
# plot_predictions(y_test_m0, result0['predictions'], model_name="PersistenceBaseline", datetime_index=test_datetime)
# plt.show()


# Questions:
# how to add a model ? What properties should it have ? like a .predict() method ? or a .fit() method ? or something else ?
# how to keep track of the loss function ? etc 

## Compare All Models


In [ ]:
# Compare all models
if len(results) > 0:
    scorecard = compare(results, plot=True)
else:
    print("No models evaluated yet. Add models in the section above.")


No models evaluated yet. Add your models in the section above.
